In [1]:
import cx_Oracle
from enum import unique
class params_global():
    is_test = "True"
    # is_test = "False"

    nodata_exec_status = 2
"""
here to save all read config for Ezpass database
"""
class ezpass_db:
    # @unique
    class Authconfig:
        if params_global.is_test:
            # 测试环境cusrc
            ods_oracle_account = 'CUSRC/easipass'
            oracle_database_dbods = '192.168.129.149:1521/test12c'
            oracle_write_account = 'CUSRC:easipass'
            oracle_write_address = '192.168.129.149:1521/?service_name=test12c'
            oracle_write_account_alarm = 'DW_CUS_RC:easipass'
            oracle_write_address_alarm = '192.168.130.225:1521/?service_name=pdbcusdev'

            # # 测试环境 dw
            dw_oracle_account = 'BI_OPER/easipass'
            oracle_database_dbdw = '192.168.129.179:1521/testdw'

            # 测试环境BD_RISK
            ods_oracle_account = 'BD_RISK/easipass'
            oracle_database_dbods = '192.168.129.149:1521/test12c'
            # oracle_write_account = 'BD_RISK:easipass'
            # oracle_write_address = '192.168.129.149:1521/?service_name=test12c'

            dm_oracle_account = 'BD_RISK/infa4A9F'
            oracle_database_dbdm = '192.168.132.52:1521/dbdm'

            gold_oracle_account = 'n_epz_cus/n_epz_cus'
            oracle_database_gold = '192.168.131.125:1521/zmqwdb'

            alarm_oracle_account = 'DW_CUS_RC/easipass'
            oracle_database_alarm = '192.168.130.225:1521/pdbcusdev'

            lgsa_oracle_account = 'lgsa/lgsa'
            oracle_database_lgsa = '192.168.131.125:1521/zmqwdb'

            testlog_oracle_account = 'BD_RISK/testinfaer'
            oracle_database_testlog = '192.168.129.163:1521/TESTDMAPP'
        else:
            # 生产环境 CUSRC
            # ods_oracle_account = 'CUSRC/infa5F1D'
            # oracle_database_dbods = '192.168.132.12:1521/dbods'
            # oracle_write_account = 'CUSRC:infa5F1D'
            # oracle_write_address = '192.168.132.12:1521/?service_name=dbods'

            # 生产环境 BD_RISK
            oracle_write_account = 'BD_RISK:infa4A9F'
            oracle_write_address = '192.168.132.52:1521/?service_name=dbdm'
            # dbods
            ods_oracle_account = 'CUSRC/infa5F1D'
            oracle_database_dbods = '192.168.132.12:1521/dbods'
            # db_dw
            dw_oracle_account = 'BI_OPER/infa6D83'
            oracle_database_dbdw = '192.168.132.17:1521/dbdw'
            # db_dm
            dm_oracle_account = 'BD_RISK/infa4A9F'
            oracle_database_dbdm = '192.168.132.52:1521/dbdm'

            # Mongodb_config
            mongo_read_oper = 'ods_sens_oper:mgoD52F'
            mongo_ip = '192.168.132.74:27017'
            mongo_database = 'ods_sens_zmxpq'
            mongo_table = 'credit'
# from loguru import logger
import time
import pandas as pd
class Read_Oracle(object):
    def __init__(self):
        self.link = None
        self.account = None

    def read_oracle_df(self, sql):
        print(self.account)
        print(self.link)
        conn = cx.connect(self.account+'@'+self.link, encoding='utf8')
        data_gen = pd.read_sql(con=conn, sql=sql, chunksize=100000)
        gen_list = []
        for gen in data_gen:
            gen_list.append(gen)
        if len(gen_list) ==0:
            print('Sql cannot find table, please check database')
            data_df = None
        else:
            data_df = pd.concat(gen_list)
        return data_df


    def read_oracle(self, sql, database):
        start_time = time.time()
        """
        :arg
            :param sql : 读取的select 脚本
            :param databse ：目标database ； dbods、dbdw、dbdm
        :return
            返回查询的结果dataframe
        """
        sql = sql
        if database == 'dbods':
            self.link = ezpass_db.Authconfig.oracle_database_dbods
            self.account = ezpass_db.Authconfig.ods_oracle_account
        elif database == 'dbdw':
            self.link = ezpass_db.Authconfig.oracle_database_dbdw
            self.account = ezpass_db.Authconfig.dw_oracle_account
        elif database == 'dbdm':
            self.link = ezpass_db.Authconfig.oracle_database_dbdm
            self.account = ezpass_db.Authconfig.dm_oracle_account
        elif database == 'dbgold':
            self.link = ezpass_db.Authconfig.oracle_database_gold
            self.account = ezpass_db.Authconfig.gold_oracle_account
        elif database == 'dbalarm':
            self.link = ezpass_db.Authconfig.oracle_database_alarm
            self.account = ezpass_db.Authconfig.alarm_oracle_account
        elif database == 'dblgsa':
            self.link = ezpass_db.Authconfig.oracle_database_lgsa
            self.account = ezpass_db.Authconfig.lgsa_oracle_account
        elif database == 'testlog':
            self.link = ezpass_db.Authconfig.oracle_database_testlog
            self.account = ezpass_db.Authconfig.testlog_oracle_account
        else:
            print('read_oracle config error, please match your input sql and db')
            return

        data_df = self.read_oracle_df(sql)
        if data_df is None:
            data_df = pd.DataFrame()
        if data_df is not None:
            data_df = data_df.fillna('N/A')
        end_time = time.time()
        consum_time = end_time - start_time
        print('Read Table successfully! , Total read time spent {}s'.format(str(consum_time)[:5]))
        return data_df
def deal_context_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['businessStartTime'] = datetime.datetime.now()
    temp_dic['businessEndTime'] = datetime.datetime.now()
    temp_dic['entry'] = df['entry']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "}"
def deal_entry_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['entryId'] = df['ENTRY_ID']
    temp_dic['entryIdCompare'] = df['ENTRY_ID_COMPARE']
    temp_dic['comparison'] = df['COMPARISON']
    temp_dic['ownerCode'] = df['CONSIGN_SCC']
    temp_dic['ownerName'] = df['CONSIGN_NAME']
    temp_dic['ownerCodeCompare'] = df['CONSIGN_SCC_COMPARE']
    temp_dic['ownerNameCompare'] = df['CONSIGN_NAME_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"
def deal_comparison_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['code_t'] = df['G_NAME']
    temp_dic['qty1'] = df['G_QTY']
    temp_dic['originCountry'] = df['ORIGIN_COUNTRY']
    temp_dic['codeTCompare'] = df['G_NAME_COMPARE']
    temp_dic['qty1Compare'] = df['G_QTY_COMPARE']
    temp_dic['originCountryCompare'] = df['ORIGIN_COUNTRY_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"
import cx_Oracle as cx
import pandas as pd
import os
import datetime
from loguru import logger
import time
pd.set_option('display.max_columns', None)
from config.auth_config import ezpass_db
os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'
def deal_context_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['businessStartTime'] = datetime.datetime.now()
    temp_dic['businessEndTime'] = datetime.datetime.now()
    temp_dic['entry'] = df['entry']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "}"
def deal_entry_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['entryId'] = df['ENTRY_ID']
    temp_dic['entryIdCompare'] = df['ENTRY_ID_COMPARE']
    temp_dic['comparison'] = df['COMPARISON']
    temp_dic['ownerCode'] = df['CONSIGN_SCC']
    temp_dic['ownerName'] = df['CONSIGN_NAME']
    temp_dic['ownerCodeCompare'] = df['CONSIGN_SCC_COMPARE']
    temp_dic['ownerNameCompare'] = df['CONSIGN_NAME_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"
def deal_comparison_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['code_t'] = df['G_NAME']
    temp_dic['qty1'] = df['G_QTY']
    temp_dic['originCountry'] = df['ORIGIN_COUNTRY']
    temp_dic['codeTCompare'] = df['G_NAME_COMPARE']
    temp_dic['qty1Compare'] = df['G_QTY_COMPARE']
    temp_dic['originCountryCompare'] = df['ORIGIN_COUNTRY_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"

In [2]:
import cx_Oracle as cx
import pandas as pd
import os
from loguru import logger
from sqlalchemy import types, create_engine
from risk_models.config.auth_config import ezpass_db
from risk_models.config.read_config.read_func import Read_Oracle
import datetime
import time
from sqlalchemy.types import *

os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'


class Write_Oracle_Alarm(object):
    def __init__(self):
        # TODO 需要修改
        self.dmlink = ezpass_db.Authconfig.oracle_database_dbdm
        self.dwlink = ezpass_db.Authconfig.oracle_database_dbdw
        self.odslink = ezpass_db.Authconfig.oracle_database_dbods
        self.account = ezpass_db.Authconfig.ods_oracle_account
        self.write_account = ezpass_db.Authconfig.oracle_write_account_alarm
        self.write_address = ezpass_db.Authconfig.oracle_write_address_alarm
        # 目前只考虑一个conn，以后有多个写入地址再加
        self.conn = create_engine(f"oracle+cx_oracle://{self.write_account}@{self.write_address}",
                                  encoding='utf-8', convert_unicode=True)
        self.conn.autocommit = True

    def get_max_index(self, table_name):
        """
        :param table_name: 旧表表名
        :return: 返回最大的ID
        """
        read_sql = f'select max("ID") as id from {table_name.lower()}'
        # todo 这里的读取用到了dbdm是因为切换了生产环境，原本是dbods
        s_id = Read_Oracle().read_oracle(sql=read_sql, database='dbalarm')
        if s_id is None:
            return 0
        else:
            return 0 if s_id['ID'].values[0] == 'N/A' else s_id['ID'].values[0]

    def get_first_line(self, table_name):
        read_sql = f'select * from {table_name.lower()} where rownum = 1 '
        # todo 这里的读取用到了dbdm是因为切换了生产环境，原本是dbods
        old_df = Read_Oracle().read_oracle(sql=read_sql, database='dbdm')
        return old_df

    def clean_index(self, new_df, old_table_name):
        """
        清洗newdf的index
        :param new_df:
        :param old_table_name:
        :return:
        """
        new_df = new_df.reset_index()
        new_df = new_df.drop(axis=1, columns=['ID']).rename(columns={'index': 'ID'})
        max_id = self.get_max_index(old_table_name)
        if max_id is not None:
            new_df.loc[:, 'ID'] = new_df.loc[:, 'ID'].apply(lambda x: x + max_id + 1)
        else:
            pass
        new_df.set_index('ID')

        return new_df

    def add_lastupdate(self, new_df):
        now_time = datetime.datetime.now()
        now_time = datetime.datetime.strftime(now_time, format='%Y-%m-%d %H:%M:%S')
        new_df.loc[:, 'LASTUPDATE'] = now_time
        new_df.loc[:, 'LASTUPDATE'] = pd.to_datetime(new_df.loc[:, 'LASTUPDATE'], format='%Y-%m-%d %H:%M:%S')

        return new_df

    def add_iscurrent(self, new_df):
        new_df.loc[:, 'ISCURRENT'] = 1
        return new_df

    def change_lastupdate(self, old_table_name, org_code, alarm):
        """
        :param old_table_name: 表名
        :param org_code: 企业信用代码
        :param alarm: [model_code, child_mode_code] 主模块id和子模块id
        :return: None
        """
        now_time = datetime.datetime.now()
        now_time = datetime.datetime.strftime(now_time, format='%Y-%m-%d %H:%M:%S')
        if org_code is not None and alarm is None:
            try:
                self.conn.execute(
                    "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and org_code ='{}'".format(
                        old_table_name, now_time, org_code))
            except Exception as e:
                logger.error(e)
        elif alarm is not None:
            if org_code is not None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and org_code ='{}' and model_code = '{}'".format(
                                old_table_name, now_time, org_code, model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and org_code ='{}' and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, now_time, org_code, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
            elif org_code is None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and model_code = '{}'".format(
                                old_table_name, now_time,  model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, now_time, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
        elif org_code is None:
            try:
                self.conn.execute(
                    "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 ".format(old_table_name, now_time))
            except Exception as e:
                logger.error(e)
        else:
            logger.warning('change_lastupdate func params error, please check params')

    def change_iscurrent(self, old_table_name, org_code, alarm):
        if org_code is not None and alarm is None:
            try:
                self.conn.execute("update {} set ISCURRENT = 0  where iscurrent = 1 and org_code ='{}'".format(old_table_name, org_code))
            except Exception as e:
                logger.error(e)
        elif alarm is not None:
            if org_code is not None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0  where iscurrent = 1 and org_code ='{}' and model_code = '{}'".format(old_table_name, org_code, model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0  where iscurrent = 1 and org_code ='{}' and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, org_code, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
            elif org_code is None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0 where iscurrent = 1 and model_code = '{}'".format(old_table_name, model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0 where iscurrent = 1 and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
        elif org_code is None:
            try:
                self.conn.execute("update {} set ISCURRENT = 0 where iscurrent = 1 ".format(old_table_name))
            except Exception as e:
                logger.error(e)
        else:
            logger.warning('change_iscurrent_func params error, please check params')

    @logger.catch()
    def write_oracle(self, table_name, new_df, org_code, alarm, iscurrent=True, alarm_code = None):
        """
        风控产品封装的写入接口
        :param iscurrent: 是否需要在插入的时候更新iscurrent
        :param table_name: 需要写入的表名
        :param new_df: 写入的对象dataframe
        :param org_code: 需要更新的企业信用代码
        :param alarm:  [model_code, child_mode_code] 主模块id和子模块id
        :return: None
        """
        if new_df is None:
            return
        start_time = time.time()
        new_df = self.clean_index(new_df, table_name)
        print(new_df)
        new_df = self.add_lastupdate(new_df)
        new_df = self.add_iscurrent(new_df)
        self.change_lastupdate(table_name, org_code, alarm)
        if iscurrent:
            self.change_iscurrent(table_name, org_code, alarm)
        # old_df = self.get_first_line(table_name)

        for key, value in dict(new_df.dtypes).items():
            # 如果不对object的columns进行一次astype（str）就会报错
            if value == 'object':
                new_df.loc[:, '{}'.format(key)] = new_df.loc[:, '{}'.format(key)].astype(str)

        def set_d_type_dict(df):
            type_dict = {}
            for i, j in zip(df.columns, df.dtypes):
                if "object" in str(j):
                    type_dict.update({i: VARCHAR(512)})
                if "float" in str(j):
                    type_dict.update({i: DECIMAL(20, 5)})
                if "int" in str(j):
                    type_dict.update({i: DECIMAL(20)})
            return type_dict

        logger.info('Processing... Writing {} rows into database'.format(len(new_df)))
        d_type = set_d_type_dict(new_df)
        new_df.to_sql(name=table_name.lower(), con=self.conn, if_exists='append', index=False, dtype=d_type, chunksize=2000)

        end_time = time.time()
        consum_time = end_time - start_time
        logger.info('Insert data into {} successfully! Total write time spent {}s'.format(table_name.upper(), str(consum_time)[:5]))


In [18]:
 sql_text = f'''
        select
        t1.ENTRY_ID,
        t1.D_DATE,
        t3.WH_REC_PREENT_NO,
        t1.OWNER_CODE_SCC,
        t1.OWNER_CODE,
        t1.OWNER_NAME,
        t1.CONSIGN_SCC,
        t1.CONSIGN_CODE,
        t1.CONSIGN_NAME,
        t1.NOTE_S
        from
        (select * from ODS_HEPS.ENTRY_HEAD where NOTE_S LIKE '%期货保税交割%') t1
        left join
        DW_CUS_RC.TRADE t2
        on t1.OWNER_CODE_SCC = t2.CREDIT_CODE and t2.LABEL_CODE LIKE '%001%'
        left join
        DW_CUS_RC.WAREHOUSE_RECEIPT_BSC t3
        on t1.ENTRY_ID = t3.RLT_ENTRY_NO
        '''
future_warehouse_df = Read_Oracle().read_oracle(sql=sql_text, database='dbalarm')

print(future_warehouse_df)

DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
2022-06-08 22:44:23.658 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 0.938s


             ENTRY_ID     D_DATE      WH_REC_PREENT_NO      OWNER_CODE_SCC  \
0  223120190004166954 2022-04-26  000621WH200730002389  91310000132612172J   
1  224920170000054090 2022-04-19                   N/A  91310000132612172J   

   OWNER_CODE  OWNER_NAME         CONSIGN_SCC CONSIGN_CODE    CONSIGN_NAME  \
0  3122S600J4  上海飞机制造有限公司  91310000132612172J   3122S600J4      上海飞机制造有限公司   
1  3122S600J4  上海飞机制造有限公司  91310000785178414H   3122S40004  施洁医疗技术（上海）有限公司   

    NOTE_S  
0  期货保税交割1  
1   期货保税交割  


In [19]:
future_warehouse_df

,ENTRY_ID,D_DATE,WH_REC_PREENT_NO,OWNER_CODE_SCC,OWNER_CODE,OWNER_NAME,CONSIGN_SCC,CONSIGN_CODE,CONSIGN_NAME,NOTE_S
0,223120190004166954,2022-04-26,000621WH200730002389,91310000132612172J,3122S600J4,上海飞机制造有限公司,91310000132612172J,3122S600J4,上海飞机制造有限公司,期货保税交割1
1,224920170000054090,2022-04-19,N/A,91310000132612172J,3122S600J4,上海飞机制造有限公司,91310000785178414H,3122S40004,施洁医疗技术（上海）有限公司,期货保税交割


In [20]:
import time
import datetime
if future_warehouse_df is not None and len(future_warehouse_df) != 0:
    future_warehouse_df['ID'] = 0
    future_warehouse_df['D_DATE'] = future_warehouse_df['D_DATE'].map(lambda x: x.strftime("%Y%m%d"))

In [11]:
future_warehouse_df['ID'] = 0

In [22]:
future_warehouse_df

,ENTRY_ID,D_DATE,WH_REC_PREENT_NO,OWNER_CODE_SCC,OWNER_CODE,OWNER_NAME,CONSIGN_SCC,CONSIGN_CODE,CONSIGN_NAME,NOTE_S,ID
0,223120190004166954,20220426,000621WH200730002389,91310000132612172J,3122S600J4,上海飞机制造有限公司,91310000132612172J,3122S600J4,上海飞机制造有限公司,期货保税交割1,0
1,224920170000054090,20220419,N/A,91310000132612172J,3122S600J4,上海飞机制造有限公司,91310000785178414H,3122S40004,施洁医疗技术（上海）有限公司,期货保税交割,0


In [23]:
sql_text = f'''select ID, ENTRY_ID, D_DATE, WH_REC_PREENT_NO, OWNER_CODE_SCC, OWNER_CODE, OWNER_NAME, CONSIGN_SCC, CONSIGN_CODE, CONSIGN_NAME, NOTE_S from DW_CUS_RC.BD_RISK_CROSS_TRADE_RESULT_FUTURE_WAREHOUSE_CLEAN '''
future_warehouse_df = Read_Oracle().read_oracle(sql=sql_text, database='dbalarm')

DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
2022-06-08 22:53:43.832 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 0.973s


In [26]:
future_warehouse_df

,ID,ENTRY_ID,D_DATE,WH_REC_PREENT_NO,OWNER_CODE_SCC,OWNER_CODE,OWNER_NAME,CONSIGN_SCC,CONSIGN_CODE,CONSIGN_NAME,NOTE_S,WARNING_TIME
0,4,223120190004166954,20220426,000621WH200730002389,91310000132612172J,3122S600J4,上海飞机制造有限公司,91310000132612172J,3122S600J4,上海飞机制造有限公司,期货保税交割1,2022-06-08 23:04:02.255800
1,1,221620200000007959,20220319,N/A,913100006929421297,3122S60004,上海元初国际物流有限公司,91310000785178414H,2,施洁医疗技术（上海）有限公司,期货保税交割,2022-06-08 23:04:02.255800
2,3,221620200000007959,20220319,N/A,913100006929421297,3122S60004,上海元初国际物流有限公司,91310000785178414H,2,施洁医疗技术（上海）有限公司,期货保税交割,2022-06-08 23:04:02.255800
3,2,221620200000007959,20220319,N/A,913100006929421297,3122S60004,上海元初国际物流有限公司,91310000785178414H,2,施洁医疗技术（上海）有限公司,期货保税交割,2022-06-08 23:04:02.255800


In [31]:
future_warehouse_df

,ID,ENTRY_ID,D_DATE,WH_REC_PREENT_NO,OWNER_CODE_SCC,OWNER_CODE,OWNER_NAME,CONSIGN_SCC,CONSIGN_CODE,CONSIGN_NAME,NOTE_S,WARNING_TIME
0,4,223120190004166954,20220426,000621WH200730002389,91310000132612172J,3122S600J4,上海飞机制造有限公司,91310000132612172J,3122S600J4,上海飞机制造有限公司,期货保税交割1,2022-06-08 23:04:02.255800
1,1,221620200000007959,20220319,N/A,913100006929421297,3122S60004,上海元初国际物流有限公司,91310000785178414H,2,施洁医疗技术（上海）有限公司,期货保税交割,2022-06-08 23:04:02.255800
2,3,221620200000007959,20220319,N/A,913100006929421297,3122S60004,上海元初国际物流有限公司,91310000785178414H,2,施洁医疗技术（上海）有限公司,期货保税交割,2022-06-08 23:04:02.255800
3,2,221620200000007959,20220319,N/A,913100006929421297,3122S60004,上海元初国际物流有限公司,91310000785178414H,2,施洁医疗技术（上海）有限公司,期货保税交割,2022-06-08 23:04:02.255800


In [29]:
def deal_dic(df):
        temp_dic = {}
        str = "{"
        temp_dic['whRecNo'] = df['WH_REC_PREENT_NO']
        temp_dic['ownerCode'] = df['CONSIGN_SCC']
        temp_dic['ownerName'] = df['CONSIGN_NAME']
        temp_dic['noteS'] = df['NOTE_S']
        for key, value in temp_dic.items():
            str += "\"%s\":\"%s\"," % (key, value)
        return str[: -1] + "}"
collect_df = pd.DataFrame()
collect_df["ID"] = 0
collect_df["CONTEXT"] = future_warehouse_df.apply(lambda x: deal_dic(x), axis=1)  #序列化
collect_df["CORP_CREDIT_CODE"] = future_warehouse_df["OWNER_CODE_SCC"]
collect_df["CUSTOMS_CODE"] = "2249"
collect_df["TYPE_FIRST"] = "GNYTJKYJ"
collect_df["TYPE_SECOND"] = "BSFW"
collect_df["BUSINESS_TYPE"] = "63"
collect_df["ORDER_TYPE"] = "entry"
collect_df["BUSINESS_NO"] = future_warehouse_df["ENTRY_ID"]
collect_df["RESOLVE_STATUS"] = "0"
collect_df["RISK_LEVEL"] = "3"
# collect_df["LABEL"] = "期货保税交割预警"
collect_df["CORP_NAME"] = future_warehouse_df["OWNER_NAME"]
collect_df["TRADE_CODE"] = future_warehouse_df["OWNER_CODE"]
print(collect_df)

   ID                                            CONTEXT    CORP_CREDIT_CODE  \
0 NaN  {"whRecNo":"000621WH200730002389","ownerCode":...  91310000132612172J   
1 NaN  {"whRecNo":"N/A","ownerCode":"9131000078517841...  913100006929421297   
2 NaN  {"whRecNo":"N/A","ownerCode":"9131000078517841...  913100006929421297   
3 NaN  {"whRecNo":"N/A","ownerCode":"9131000078517841...  913100006929421297   

  CUSTOMS_CODE TYPE_FIRST TYPE_SECOND BUSINESS_TYPE ORDER_TYPE  \
0         2249   GNYTJKYJ        BSFW            63      entry   
1         2249   GNYTJKYJ        BSFW            63      entry   
2         2249   GNYTJKYJ        BSFW            63      entry   
3         2249   GNYTJKYJ        BSFW            63      entry   

          BUSINESS_NO RESOLVE_STATUS RISK_LEVEL     CORP_NAME  TRADE_CODE  
0  223120190004166954              0          3    上海飞机制造有限公司  3122S600J4  
1  221620200000007959              0          3  上海元初国际物流有限公司  3122S60004  
2  221620200000007959              0    

In [30]:
collect_df

,ID,CONTEXT,CORP_CREDIT_CODE,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,BUSINESS_NO,RESOLVE_STATUS,RISK_LEVEL,CORP_NAME,TRADE_CODE
0,NaN,"{""whRecNo"":""000621WH200730002389"",""ownerCode"":...",91310000132612172J,2249,GNYTJKYJ,BSFW,63,entry,223120190004166954,0,3,上海飞机制造有限公司,3122S600J4
1,NaN,"{""whRecNo"":""N/A"",""ownerCode"":""9131000078517841...",913100006929421297,2249,GNYTJKYJ,BSFW,63,entry,221620200000007959,0,3,上海元初国际物流有限公司,3122S60004
2,NaN,"{""whRecNo"":""N/A"",""ownerCode"":""9131000078517841...",913100006929421297,2249,GNYTJKYJ,BSFW,63,entry,221620200000007959,0,3,上海元初国际物流有限公司,3122S60004
3,NaN,"{""whRecNo"":""N/A"",""ownerCode"":""9131000078517841...",913100006929421297,2249,GNYTJKYJ,BSFW,63,entry,221620200000007959,0,3,上海元初国际物流有限公司,3122S60004
